In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
import torch

In [ ]:
# 읽어오기
df = pd.read_csv(r"/content/drive/MyDrive/2023_산학협력_메카솔루션/미팅 자료 (우리가 작성한 파일들)/230627/크롤링_2/ns_reviewss2.csv"
                  ,encoding='utf-8', on_bad_lines='skip', low_memory=False)

# 2023년 이전 리뷰 삭제
df['year'] = df['time'].str[:4]
#df = df[(df['year'] == '2023') | (df['product_category'].isin(['발각질제거기', '로잉머신', '거실매트']))]

df = df[((df['year'] == '2022') & (df['product_category'].isin(['발각질제거기', '로잉머신', '거실매트']))) | (df['year'] == '2023')]
#df = df[(df['year'] =='2023')]
# 카테고리, 별점, 리뷰 제외 데이터 없애기
grouped = df.drop(['userid', 'review_ranking_score', 'time'], axis=1)

# 중복 리뷰 삭제
grouped.drop_duplicates(subset='content', inplace=True)

# 별점이 제대로 저장 안되었으면 삭제
grouped = grouped[(grouped['star_rating'] >= 1) & (grouped['star_rating'] <= 5)]

# grouping
grouped = grouped.groupby(['product_category', 'productName', 'star_rating'])['content'].apply(' '.join).reset_index()

# 공백, 한글, 알파벳, 숫자 제외 없애기
grouped['content'] = grouped['content'].apply(lambda x: re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣A-Za-z0-9 ]+', '', str(x)))

In [1]:
# 모델 불러오기

label_mapping = {'0': 'NEGATIVE', '1': 'POSITIVE'}
num_labels = len(label_mapping)

tokenizer = PreTrainedTokenizerFast.from_pretrained("ainize/kobart-news")
model = BartForConditionalGeneration.from_pretrained("ainize/kobart-news")

# Set the device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 요약 컬럼 만들기
grouped['summary'] = ''
grouped['summary_tokens'] = ''

#input_ids = tokenizer.encode(review, return_tensors="pt")

#max_input_length = tokenizer.model_max_length
#max_input = model.vocab_size
#print(model.config.max_position_embeddings)

NameError: ignored

In [ ]:
# 각 아이템 상품 별점 리뷰 당 토큰 length 출력

for index, row in grouped.iterrows():
    product_category = row['product_category']
    star_rating = row['star_rating']
    productName = row['productName']
    review = row['content']

    # Encode
    input_ids = tokenizer.encode(review, return_tensors="pt")

    length = len(input_ids[0])

    print("Product Category:", product_category, end=" | ")
    print("Star Rating:", star_rating, end=" | ")
    print("Product Name:", productName, end=" | ")
    print("Review Length:", length)

In [ ]:
# 전체 인코딩 디코딩 과정

max_total_length = 1024
max_segments = 3

for index, row in grouped.iterrows():
    review = row['content']
    product_category = row['product_category']
    star_rating = row['star_rating']
    productName = row['productName']
    print()
    print("Product Category:", product_category, end=" | ")
    print("Star Rating:", star_rating, end=" | ")
    print("Product Name:", productName)

    # Encode
    input_ids = tokenizer.encode(review, return_tensors="pt").to(device)

    segments = []
    start_idx = 0
    length = len(input_ids[0])

    # segment 수 계산
    num_segments = min(max_segments, -(-length // max_total_length))  # Ceiling division

    # segment length 계산
    segment_length = min(max_total_length, length // num_segments)

    # Segment the input
    for i in range(num_segments):
        start_idx = i * segment_length
        end_idx = min((i + 1) * segment_length, length)
        segment_split = input_ids[:, start_idx:end_idx]
        segments.append(segment_split)

    summaries = []
    summary_tokens_list = []

    # Generate summaries for each segment
    for segment in segments:
        summary_text_ids = model.generate(
            input_ids=segment,
            bos_token_id=model.config.bos_token_id,
            eos_token_id=model.config.eos_token_id,
            length_penalty=1.2,
            max_length=128,
            min_length=32,
            num_beams=4,
            no_repeat_ngram_size=1,
            num_return_sequences=1
        )

        # Decode
        summary_tokens = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)
        print(summary_tokens)
        # Concat summaries
        summaries.append(summary_tokens)
        summary_tokens_list.append(summary_text_ids.tolist())

    # Concat
    summary = ' '.join(summaries)
    summary_tokens_str = ' '.join([str(tokens) for sublist in summary_tokens_list for tokens in sublist])

    # 업데이트
    grouped.at[index, 'summary'] = summary
    grouped.at[index, 'summary_tokens'] = summary_tokens_str



Product Category: 거실매트 | Star Rating: 1 | Product Name: 국민 아기매트 본베베 유아 국민 거실 폴더매트
구주신상품의 커버에서 본드냄새가 나네요 아기제품인데 높이가 4cm로 다른회사 보다 두껍고 저밀도 폼이라 많이 꺼졌네요. 구매확정후에도 사이즈오송 하면 빨리 처리해주셔야 좋겠습니다.
매자문의로 판매사 제품 포장영상을 확인해 보고 다시 연락처 표정이 다르고 9번째 구매상품의이 약 한곳에만 문제가 있어 매트 2개 사은품을 주려하는데 규격이 달라서 해외직접 시 2년 이라고 밝혔습니다.
경제기자가 다신주문하고싶힘  한 건의 길이 2cm 정도 차이가 납납니다. 라는 신다입니다 제 최초 사은품...전에는 공짜로 달라고한것도 아니고 시트두개 사이즈가 다른 두개의 길이 너무 크게 생겼였습니다.

Product Category: 거실매트 | Star Rating: 2 | Product Name: 국민 아기매트 본베베 유아 국민 거실 폴더매트
전직구라 주문폭주해서 배송이슈발생 아가용이 아니라 강아지용으로 샀는데 미끄러워서 위에 다시 카페트 깔고 사용해요 그라고 사람이 걸으면 삐쩍 붙는 소리가 넘적대며 봉제선도 저렴하게 반품하지 못하고 걍 돈 더 쓰고 비싼거 살걸 그랬어요ᅮᅲ

Product Category: 거실매트 | Star Rating: 3 | Product Name: 국민 아기매트 본베베 유아 국민 거실 폴더매트
구매래치 불량제품을 받게되어 바로 못사용해서 구매했던 매트리가 만족스러운 배송을 받았고 다른 제품들도 사서 편하게 사용할 수 있는 점과 몇가지 사항을 적재한 채 재구매를 진행하였기 때문에 이점이 아쉬울 뿐이라고 밝혔다.
최자혜씨가 제로캡매트 구매에 조건을 잘못 설정하여 배송이 늦어져서 죄송한 마음을 표하였는데 상품의 출고가는 2년, 판매량은 5백 미터로 짧지만 두배 정도라고 답해주었으며 후에 다른 제품의 문제점도 알려줄 수 있을 것 같아 다행이라고 소감을 남겼다.
큰자리가 너무 좋고 신상품 매트 가격이 저렴해 구매한지

In [ ]:
# 아이템으로 다시 묶음
filtered_summary = grouped.groupby(['product_category', 'star_rating'])['summary'].apply(' '.join).reset_index()

print(filtered_summary)

   product_category  star_rating  \
0              거실매트            1   
1              거실매트            2   
2              거실매트            3   
3              거실매트            4   
4              거실매트            5   
5               남자옷            1   
6               남자옷            2   
7               남자옷            3   
8               남자옷            4   
9               남자옷            5   
10              디퓨져            1   
11              디퓨져            2   
12              디퓨져            3   
13              디퓨져            4   
14              디퓨져            5   
15             로잉머신            1   
16             로잉머신            2   
17             로잉머신            3   
18             로잉머신            4   
19             로잉머신            5   
20            리클라이너            1   
21            리클라이너            2   
22            리클라이너            3   
23            리클라이너            4   
24            리클라이너            5   
25           발각질제거기            1   
26           발각질제거기         

In [ ]:
# csv 파일로 저장
filtered_summary.to_csv('after_koBART_summary.csv')
!cp after_koBART_summary.csv "drive/My Drive/"